### Test ADWH connection

* simplified from the versione inside Conda NB

In [1]:
import cx_Oracle as cxo
import ads

from ads.database import connection
from ads.database.connection import Connector
from ads.dataset.factory import DatasetFactory
from sqlalchemy import create_engine
from urllib.request import urlopen
import os
import warnings
import logging

import pandas as pd

warnings.filterwarnings("ignore", category=DeprecationWarning)
logging.basicConfig(format="%(levelname)s:%(message)s", level=logging.INFO)

In [2]:
print(cxo.__version__)

8.3.0


In [3]:
print(ads.__version__)

2.5.7


In [4]:
# credential = {'database_name': 'dbtst1_medium',
#              'username': 'DEMO1',
#              'password': 'Pippo12345::',
#              'database_type':'oracle'}

In [5]:
# memorizzo in un repository locale
# in questo modo nel NB non c'è la pwd
# che ho memorizzato prima nel repo (vedi istruzione commentata)
database_name = "dbtst1_medium"

# connection.update_repository(key=database_name, value=credential)
repo = connection.get_repository(key=database_name)

In [6]:
# import the wallet
wallet_path = os.path.join(
    os.path.expanduser("~/data-science-bp"), f"Wallet_{database_name}.zip"
)

connection.import_wallet(wallet_path=wallet_path, key=database_name)

In [7]:
if "database_name" in globals() and database_name != "<database_name>":
    connector = Connector(key=database_name)

    print(connector.config)
    print()
    print(f"Connector uri is: {connector.uri}")
else:
    print("Skipping as it appears that you do not have database_name configured.")

{'database_type': 'oracle', 'database_name': 'dbtst1_medium', 'username': 'DEMO1', 'password': 'Pippo12345::'}

Connector uri is: oracle+cx_oracle://DEMO1:Pippo12345::@dbtst1_medium


In [8]:
#
# Here: definisco la query che voglio eseguire sul DB
#
query = """
    SELECT * from CUSTOMERS where rownum < 50
    """

In [9]:
%%time

# eseguo la query e carico tutto in un Pandas DataFrame
customers_df = DatasetFactory.open(connector.uri, format="sql", table=query)

loop1:   0%|          | 0/4 [00:00<?, ?it/s]


49 rows [00:00, 4556.20 rows/s]


CPU times: user 425 ms, sys: 55.8 ms, total: 480 ms
Wall time: 753 ms


In [10]:
type(customers_df)

ads.dataset.dataset.ADSDataset

In [11]:
df = customers_df.to_pandas_dataframe()

In [12]:
type(df)

pandas.core.frame.DataFrame

In [13]:
df.tail(10)

,cust_id,cust_first_name,cust_last_name,cust_gender,cust_year_of_birth,cust_marital_status,cust_street_address,cust_postal_code,cust_city,cust_city_id,...,cust_main_phone_number,cust_income_level,cust_credit_limit,cust_email,cust_total,cust_total_id,cust_src_id,cust_eff_from,cust_eff_to,cust_valid
39,28117,Abner,Kenney,M,1957,None,87 West Covington Avenue,83967,Tilburg,52369,...,636-716-9795,"K: 250,000 - 299,999",15000,Kenney@company.example.com,Customer total,52772,None,1998-01-01,None,I
40,31672,Abner,Kenney,M,1950,single,97 Saguache Avenue,33415,Pala,52101,...,569-349-9468,"K: 250,000 - 299,999",11000,Kenney@company.example.com,Customer total,52772,None,1998-01-01,None,I
41,35228,Abner,Kenney,M,1952,married,97 South Gloucester Avenue,59862,Malaga,51894,...,537-204-8513,"K: 250,000 - 299,999",7000,Kenney@company.example.com,Customer total,52772,None,1998-01-01,None,I
42,37006,Abner,Robbinette,M,1953,single,107 East Braxton Avenue,61189,Evinston,51459,...,572-711-6909,"K: 250,000 - 299,999",11000,Robbinette@company.example.com,Customer total,52772,None,1998-01-01,None,I
43,40561,Abner,Robbinette,M,1959,married,107 South Page Avenue,48346,Noma,51985,...,159-716-9696,"K: 250,000 - 299,999",7000,Robbinette@company.example.com,Customer total,52772,None,1998-01-01,None,I
44,44116,Abner,Robbinette,M,1956,None,117 North Covington Avenue,59411,Henley-on-Thames,51659,...,527-349-4493,"K: 250,000 - 299,999",15000,Robbinette@company.example.com,Customer total,52772,None,1998-01-01,None,I
45,34359,Abner,Robbinette,M,1971,married,17 North Kaufman Court,54845,Haswell,51651,...,593-349-9980,"K: 250,000 - 299,999",11000,Robbinette@company.example.com,Customer total,52772,None,1998-01-01,None,I
46,47895,Abner,Robbinette,M,1963,married,27 West 5th Boulevard,57128,Barry,51166,...,379-204-8906,"K: 250,000 - 299,999",7000,Robbinette@company.example.com,Customer total,52772,None,1998-01-01,None,I
47,1451,Abner,Robbinette,M,1962,single,37 Mckenzie Street,68644,Schwaebisch Gmuend,52300,...,652-226-5722,"K: 250,000 - 299,999",15000,Robbinette@company.example.com,Customer total,52772,None,1998-01-01,None,A
48,5006,Abner,Robbinette,M,1989,None,37 South Braxton Street,82590,Warstein,52497,...,304-692-9126,"K: 250,000 - 299,999",11000,Robbinette@company.example.com,Customer total,52772,None,1998-01-01,None,A


In [14]:
print(f"The number of records read is {customers_df.shape[0]}")

The number of records read is 49


### Using directly Pandas (with our extensions)

* idea taken from: https://github.com/oracle/accelerated-data-science

In [15]:
connection_parameters = {
      "user_name": repo["username"],
      "password": repo["password"],
      "service_name": repo["database_name"],
      "wallet_location": "/home/datascience/data-science-bp/Wallet_dbtst1_medium.zip",
  }

df = pd.DataFrame.ads.read_sql(
      "SELECT * FROM CUSTOMERS",
      connection_parameters=connection_parameters,
  )

In [16]:
df.shape

(55500, 23)

#### if we want to use bind variables:

In [17]:
df = pd.DataFrame.ads.read_sql(
    """SELECT * FROM CUSTOMERS WHERE ROWNUM <= :max_rows
    """,
    bind_variables={
          "max_rows" : 20
      },
    connection_parameters=connection_parameters
  )

df.shape

(20, 23)